In [14]:
import os
from copy import deepcopy
from pathlib import Path
import random
import shutil

from tqdm import tqdm
from pprint import pprint
import pandas as pd
import numpy as np
import nibabel as nib

from bcblib.tools.general_utils import open_json, save_json
from lesseg_unet.utils import get_sex_int, get_age_int

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from lesseg_unet.utils import get_perf_seg_dict, get_perf_seg_dict_from_folders
"""
First we need to create a dictionary with the segmentation files and the performance
"""
"""
INPUTS
"""
seg_folder = '/media/chrisfoulon/HDD2/final_training_set/abnormal_5fold_segmentation/'
output_folder = '/media/chrisfoulon/HDD2/final_training_set/perf_analyses_abnormal_5fold_segmentation/'
# seg_folder = '/media/chrisfoulon/HDD2/final_training_set/controls_5fold_segmentation/'
# output_folder = '/media/chrisfoulon/HDD2/final_training_set/perf_analyses_controls_5fold_segmentation/'

In [20]:


spm_seg_folder = Path(output_folder, 'spm_segmentation')
os.makedirs(output_folder, exist_ok=True)
os.makedirs(spm_seg_folder, exist_ok=True)

keys_struct = open_json('/media/chrisfoulon/HDD2/final_training_set/cleaned_abnormal_b1000_info_dict.json')

if Path(seg_folder, '__output_image_volumes.json').exists():
    seg_dict = get_perf_seg_dict(seg_folder, keys_struct=keys_struct, relative_output_paths=False)
else:
    print('process from folders')
    seg_dict = get_perf_seg_dict_from_folders(seg_folder, keys_struct=keys_struct, relative_output_paths=False)
# print first value of the dict
pprint({k: seg_dict[k] for k in random.sample(seg_dict.keys(), 1)})
# Then we need to copy the segmentation files to a separate folder for SPM ...
output_masks_folder = Path(output_folder, 'output_masks/')
output_labels_folder = Path(output_folder, 'output_labels/')
os.makedirs(output_masks_folder, exist_ok=True)
os.makedirs(output_labels_folder, exist_ok=True)
spm_seg_dict = {}
for k in tqdm(seg_dict):
    shutil.copy(seg_dict[k]['segmentation'], output_masks_folder)
    shutil.copy(seg_dict[k]['label'], output_labels_folder)
    spm_seg_dict[k] = deepcopy(seg_dict[k])
    spm_seg_dict[k]['segmentation'] = str(Path(spm_seg_folder, Path(seg_dict[k]['segmentation']).name.split('.gz')[0]))
    nib.save(nib.load(seg_dict[k]['segmentation']), spm_seg_dict[k]['segmentation'])
print(len(seg_dict))
pprint({k: seg_dict[k] for k in random.sample(seg_dict.keys(), 1)})

save_json(Path(seg_folder, 'perf_seg_dict.json'), seg_dict)
save_json(Path(seg_folder, 'spm_perf_seg_dict.json'), spm_seg_dict)

process from folders


100%|██████████| 712/712 [00:53<00:00, 13.35it/s]


{'1.2.840.846310145.12.1.1.60692052_20190315163801_201__pref__': {'Unnamed: 0': 70,
                                                                  'b1000': '/media/chrisfoulon/HDD2/final_training_set/cleaned_abnormal_images/non_linear_co-rigid_rigid_geomean_denoise_1.2.840.846310145.12.1.1.60692052_20190315163801_201__pref___bval1000.nii.gz',
                                                                  'core_filename': 'non_linear_co-rigid_rigid_geomean_denoise_1.2.840.846310145.12.1.1.60692052_20190315163801_201__pref___bval1000_v174v',
                                                                  'dice_metric': 0.9855907559394836,
                                                                  'distance': 0.0,
                                                                  'label': '/media/chrisfoulon/HDD2/final_training_set/controls_5fold_segmentation/fold_1/val_images/R_Thalamoperforators/label_non_linear_co-rigid_rigid_geomean_denoise_1.2.840.846310145.12.1.1.60692

100%|██████████| 3563/3563 [04:36<00:00, 12.91it/s]


3563
{'wodctH25_b1000_clmp_b1000_40353637_M_19530720_20081222_SIEMENS_Avanto_15_MRHead_ep2d_diff_3scan_ipat_ep_b1000t_1': {'Unnamed: 0': 239,
                                                                                                                      'b1000': '/media/chrisfoulon/HDD2/final_training_set/cleaned_abnormal_images/wodctH25_b1000_clmp_b1000_40353637_M_19530720_20081222_SIEMENS_Avanto_15_MRHead_ep2d_diff_3scan_ipat_ep_b1000t_1.nii.gz',
                                                                                                                      'core_filename': 'wodctH25_b1000_clmp_b1000_40353637_M_19530720_20081222_SIEMENS_Avanto_15_MRHead_ep2d_diff_3scan_ipat_ep_b1000t_1_v230v',
                                                                                                                      'dice_metric': 0.7714987993240356,
                                                                                                                      'distance': 3

In [21]:
"""
Then we need to create a dataframe with the performance metrics
"""
columns=['segmentation', 'dice_metric', 'distance', 'volume', 'lesion_cluster', 'PatientAge', 'PatientSex']
b1000_info_dict = open_json('/media/chrisfoulon/HDD2/final_training_set/cleaned_abnormal_b1000_info_dict.json')


# dict_for_df = {}
# for k in seg_dict:
#     for col in columns:
#         if col in seg_dict:
#             dict_for_df[k][col] = seg_dict[k][col]
#         else:
#             dict_for_df[k][col] = b1000_info_dict[k][col]
#
# pprint({k: dict_for_df[k] for k in random.sample(dict_for_df.keys(), 1)})
# new_df = pd.DataFrame.from_records(dict_for_df).T
# new_df.to_csv(Path(seg_folder, 'segmentation_perf_df.csv'), columns=columns)

# create a function to do that
def get_df_from_dict(seg_dict, columns):
    dict_for_df = {}
    for k in seg_dict:
        dict_for_df[k] = {}
        for col in columns:
            if col in seg_dict[k]:
                if col == 'PatientAge':
                    dict_for_df[k][col] = get_age_int(seg_dict[k][col])
                elif col == 'PatientSex':
                    dict_for_df[k][col] = get_sex_int(seg_dict[k][col])
                else:
                    dict_for_df[k][col] = seg_dict[k][col]
            elif col == 'lesion_cluster':
                dict_for_df[k][col] = Path(seg_dict[k]['segmentation']).parent.name
            else:
                if col == 'PatientAge':
                    dict_for_df[k][col] = get_age_int(b1000_info_dict[k][col])
                elif col == 'PatientSex':
                    dict_for_df[k][col] = get_sex_int(b1000_info_dict[k][col])
                else:
                    dict_for_df[k][col] = b1000_info_dict[k][col]
    new_df = pd.DataFrame.from_records(dict_for_df).T
    return new_df

# use it on seg_dict and spm_seg_dict
new_df = get_df_from_dict(seg_dict, columns)
new_df.to_csv(Path(output_folder, 'segmentation_perf_df.csv'), columns=columns)
new_df = get_df_from_dict(spm_seg_dict, columns)
new_df.to_csv(Path(output_folder, 'spm_segmentation_perf_df.csv'), columns=columns)

In [22]:
"""
Then we need to zscore the columns
"""
zcores_columns = ['dice_metric', 'distance', 'volume', 'PatientAge', 'PatientSex']
for col in zcores_columns:
    # pd.to_numeric(new_df[col])
    new_df[col] = new_df[col].infer_objects()
#     print(col)
#     print(new_df[col].dtype)
#     new_df[col] = new_df[col].apply(stats.zscore)
normalized_df = new_df[zcores_columns]
normalized_df=(normalized_df-normalized_df.mean())/normalized_df.std()
normalized_df=(normalized_df-normalized_df.min())/(normalized_df.max()-normalized_df.min())
new_df[zcores_columns] = normalized_df[zcores_columns]
new_df.to_csv(Path(seg_folder, 'segmentation_perf_df_zscored.csv'), columns=columns)

In [23]:
from nilearn.image import smooth_img
# smooth all the images from the segmentation dict with key 'segmentation' and save them in '/media/chrisfoulon/HDD2/final_training_set/abnormal_5fold_segmentation_masks_smoothed_8'
smooth_out_folder = output_folder + 'masks_smoothed_8'
os.makedirs(smooth_out_folder, exist_ok=True)
spm_smoothed_seg_dict = {}
for k in tqdm(spm_seg_dict):
    smoothed_img = smooth_img(spm_seg_dict[k]['segmentation'], 8)
    spm_smoothed_seg_dict[k] = deepcopy(spm_seg_dict[k])
    spm_smoothed_seg_dict[k]['segmentation'] = str(Path(smooth_out_folder, Path(spm_seg_dict[k]['segmentation']).name))
    nib.save(smoothed_img, spm_smoothed_seg_dict[k]['segmentation'])



100%|██████████| 3563/3563 [03:59<00:00, 14.88it/s]


In [27]:
new_df = get_df_from_dict(spm_smoothed_seg_dict, columns)
new_df.to_csv(Path(output_folder, 'spm_smoothed_segmentation_perf_df.csv'), columns=columns)